In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
# from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from cascade_rcnn import cascade_resnet50_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
# model = cascade_resnet50_fpn(pretrained=False, min_size=1024, max_size=1024)

In [3]:
import easydict 
args = easydict.EasyDict({ "batch_size": 4, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':2,
                         'output_dir' :'../trained_model/cascadedrcnn_resnet50_fpn/'})

In [4]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [5]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [6]:
num_classes = 2
model = cascade_resnet50_fpn(pretrained=True, min_size=1024, max_size=1024)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
in_features = model.roi_heads2.box_predictor.cls_score.in_features
model.roi_heads2.box_predictor = FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

trainable_backbone_layers  3
model is loaded to gpu


In [7]:
# from torch.hub import load_state_dict_from_url
# model_urls = {
#     'fasterrcnn_resnet50_fpn_coco':
#         'https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth',
#     'fasterrcnn_mobilenet_v3_large_320_fpn_coco':
#         'https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth',
#     'fasterrcnn_mobilenet_v3_large_fpn_coco':
#         'https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth'
# }
# state_dict = load_state_dict_from_url(model_urls["fasterrcnn_resnet50_fpn_coco"], progress=False)

In [8]:
# checkpoint = torch.load('../trained_model/cascadedrcnn_resnet50_fpn/checkpoint.pth')
# model.load_state_dict(checkpoint['model'])

In [9]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [ ]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(120):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=00)
    lr_scheduler.step()
    
    if epoch > 80 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

        # evaluate after every epoch
    if epoch > 50 and epoch % 5 == 0 :
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/754]  eta: 0:15:37  lr: 0.001000  loss: 6.1467 (6.1467)  loss_classifier: 0.3744 (0.3744)  loss_box_reg: 0.0083 (0.0083)  loss_objectness: 5.0485 (5.0485)  loss_rpn_box_reg: 0.7155 (0.7155)  time: 1.2432  data: 0.7607  max mem: 4858
Epoch: [0]  [753/754]  eta: 0:00:00  lr: 0.001000  loss: 0.9716 (1.0040)  loss_classifier: 0.1112 (0.0982)  loss_box_reg: 0.0993 (0.0900)  loss_objectness: 0.3644 (0.4513)  loss_rpn_box_reg: 0.3360 (0.3645)  time: 0.4719  data: 0.0093  max mem: 5123
Epoch: [0] Total time: 0:06:01 (0.4790 s / it)
Epoch: [1]  [  0/754]  eta: 0:14:59  lr: 0.001000  loss: 1.0970 (1.0970)  loss_classifier: 0.1408 (0.1408)  loss_box_reg: 0.1485 (0.1485)  loss_objectness: 0.4594 (0.4594)  loss_rpn_box_reg: 0.3483 (0.3483)  time: 1.1926  data: 0.7262  max mem: 5123
Epoch: [1]  [753/754]  eta: 0:00:00  lr: 0.001000  loss: 0.8421 (0.9369)  loss_classifier: 0.1003 (0.1045)  loss_box_reg: 0.0978 (0.0993)  loss_objectness: 0.3199 (0.4049)  loss_rpn_box_reg: 0.3096 (0.32

In [ ]:
# proposals = list(li['boxes'] for li in detection1)
# proposals

In [ ]:
evaluate(model, test_loader, device=device) 